# all about logging

related: https://realpython.com/python-logging/

**BEST PRACTICES:**
* Log at the closest point to where you can handle the error meaningfully
* Don't log the same error multiple times

## getting familiar with the logging module

**logging levels**

| Constant | Numeric Value| String Value
|---|---|---|
| logging.DEBUG	|10|	DEBUG
| logging.INFO	|20|	INFO
| logging.WARNING	|30|	WARNING
| logging.ERROR	|40|	ERROR
| logging.CRITICAL|	50|	CRITICAL

Calling **basicConfig()** to configure the root logger only works if the root logger hasn’t been configured before. All logging functions automatically call basicConfig() without arguments if basicConfig() has never been called. So, for example, once you call logging.debug(), you’ll no longer be able to configure the root logger with basicConfig().

This is why i need to restart the kernel when i want to try differnt things

**basicConfig()** contains a lot of [predefined attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes)

i can use **printf** style string formatting. but i can also use dollar sign and **curly braces** (but then i need to configure the style argument)

In [1]:
import logging

logging.basicConfig(format="{levelname}:{name}:{message}", style="{")
logging.warning("Hello, Warning!")

In [2]:
import logging

logging.basicConfig(
    format="{asctime} - {levelname} - {message}",
    style="{",
    datefmt="%Y-%m-%d %H:%M:%S",  # i can also not show the ms see more here: https://docs.python.org/3/library/time.html#time.strftime
)
logging.warning("Hello, Warning!")

### logging to a file

In [3]:
import logging

logging.basicConfig(
    filename="app.log",
    encoding="utf-8",
    filemode="a",  # a is for append, w is for write
    format="{asctime} - {levelname} - {message}",
    style="{",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logging.warning("Hello, Warning!")

### logging errors

In [4]:
import logging

logging.basicConfig(
    format="{asctime} - {levelname} - {message}",
    style="{",
    datefmt="%Y-%m-%d %H:%M:%S",
)

donuts = 5
guests = 0

try:
    donuts_per_guest = donuts / guests
except ZeroDivisionError as e:
    # setting this to False, doesn't include the actual error
    # exc_info=True can be used in any log level
    logging.error("DonutCalculationError", exc_info=True)

    # that gives you much more info than something like this:
    logging.error(f"DonutCalculationError: {e}", exc_info=False)

    # This is basically the same as the first... but shorter
    logging.exception("DonutCalculationError")

ERROR:root:DonutCalculationError
Traceback (most recent call last):
  File "/tmp/ipykernel_2207/134976379.py", line 12, in <module>
    donuts_per_guest = donuts / guests
                       ~~~~~~~^~~~~~~~
ZeroDivisionError: division by zero
ERROR:root:DonutCalculationError: division by zero
ERROR:root:DonutCalculationError
Traceback (most recent call last):
  File "/tmp/ipykernel_2207/134976379.py", line 12, in <module>
    donuts_per_guest = donuts / guests
                       ~~~~~~~^~~~~~~~
ZeroDivisionError: division by zero


## Creating custom logger

**this is the way to go**, because if you configure logging on with basicConfig, external libraries will also start outputting logs...

In [5]:
# best practice, so that the name is the moduls name in the python package namespace
import logging

logger = logging.getLogger(__name__)
logger.warning("Look at my logger!")

# we cannot use the basicConfig for a custom logger... this only the root logger gets
# for custom loggers we need to work with handlers and formatters

### Handlers

In [6]:
# Note: A logger that you create can have one or more handlers.
# That means you can send your logs to multiple places when they’re generated.

import logging

logger = logging.getLogger(__name__)
console_handler = (
    logging.StreamHandler()
)  # the logs are written to the standard error stream (stderr)
file_handler = logging.FileHandler(
    "app.log", mode="a", encoding="utf-8"
)  # sends to file

logger.addHandler(console_handler)
logger.addHandler(file_handler)
logger.handlers  # in the parentheis you can see the handlers log level

logger.warning("Look at my logger!")

Look at my logger!


The logging module comes with many handy handlers for specific purposes. For example, `RotatingFileHandler`, which creates a new log file once a file size limit is reached, or `TimedRotatingFileHandler`, with which you can create a new log file for defined intervals.

### Formatters

In [ ]:
# with the handlers you define where you send the log messages
# with the formatters you define how the log messages look like

import logging

logger = logging.getLogger(__name__)
console_handler = (
    logging.StreamHandler()
)  # the logs are written to the standard error stream (stderr)
file_handler = logging.FileHandler("app.log", mode="a", encoding="utf-8")
logger.addHandler(console_handler)
logger.addHandler(file_handler)

formatter = logging.Formatter(
    "{asctime} - {levelname} - {message}",
    style="{",
    datefmt="%Y-%m-%d %H:%M:%S",
)
console_handler.setFormatter(formatter)  # we add this to the handler, not the logger

logger.warning("Testing formatters")

Testing formatters
2025-01-10 15:46:12 - WARNING - Testing formatters


### setting log levels

In [8]:
import logging

logger = logging.getLogger(__name__)
print(logger.level)  # 0 stands for not set
print(logger)  # we see warning here... that is inherited from the root logger
print(logger.parent)
print(logger.getEffectiveLevel())  # 30 stands for warning (see table of log levels)

0
<Logger __main__ (WARNING)>
<RootLogger root (WARNING)>
30


In [9]:
# any handler added to the logger will recognize this log level

logger.setLevel("INFO")  # equal to logger.setLevel(20)
logger

<Logger __main__ (INFO)>

Note: You define the **lowest allowed log level** on the **logger itself**. Handlers can’t show logs lower than the defined log level of the logger they’re connected to.

this is why the file shows only warning and error, while the console shows also the debug

In [10]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel("DEBUG")
formatter = logging.Formatter("{levelname} - {message}", style="{")

console_handler = logging.StreamHandler()
console_handler.setLevel("DEBUG")
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

file_handler = logging.FileHandler("app.log", mode="a", encoding="utf-8")
file_handler.setLevel("WARNING")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)


logger.debug("This is a debug message")
logger.warning("This is a warning message")
logger.error("This is an error message")

This is a debug message
2025-01-10 15:46:12 - DEBUG - This is a debug message
DEBUG - This is a debug message
DEBUG:__main__:This is a debug message
This is a warning message
2025-01-10 15:46:12 - WARNING - This is a warning message
WARNING - This is a warning message
This is an error message
2025-01-10 15:46:12 - ERROR - This is an error message
ERROR - This is an error message
ERROR:__main__:This is an error message


if you want only the messages for a specific level, you can add filters to a handler. But in most cases, that is not necessary

## Logging to Azure

For this, use the container app environment logging functionality and direct the logging to a low analytics workspace.

## Logging Filtering

Filter is like a gatekeeper: it decides, whether the log gets though or is dropped. Therefore a filter returns a bool - True for allowing the log and False for dropping it.

In [8]:
import logging


# Step 1: Define a filter
class ContextFilter(logging.Filter):
    def filter(self, record):
        # Add a custom attribute
        record.request_id = "id123"
        record.logger_name = record.name
        return True  # always allow through


# Step 2: Set up logger
logger = logging.getLogger("demo")
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler()
formatter = logging.Formatter("%(levelname)s [%(request_id)s]: %(message)s")
handler.setFormatter(formatter)

# Attach filter to the handler
handler.addFilter(ContextFilter())
logger.addHandler(handler)

In [7]:
# Step 3: Log something
logger.error("hi")

ERROR [id123]: hi
ERROR [id123]: hi


In [10]:
logging.getLogger()

<RootLogger root (WARNING)>